# なんかのこびとをつくる

In [1]:
# Here is the updated code to include the new site for scraping:

import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Ensure directories exist
if not os.path.isdir("pic"):
    os.makedirs("pic")
if not os.path.isdir("name"):
    os.makedirs("name")

# URLs to scrape
urls = ["https://wikiwiki.jp/kobito/", "https://www.kobitos.com/kobitos", "https://www.google.com/search?tbm=isch&q=%E3%81%93%E3%81%B3%E3%81%A8%E3%81%A5%E3%81%8B%E3%82%93"]

for url in urls:
    # Send a GET request
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Save text data
    text = soup.get_text()
    with open(os.path.join("name", f"{url.replace('/', '_')}_text.txt"), "w", encoding='utf-8') as f:
        f.write(text)

    # Save image data
    for img in soup.find_all("img"):
        img_url = img.attrs.get("src")
        if img_url is None:
            continue
        if ".jpg" in img_url or ".webp" in img_url:
            # If the image url is relative, prepend the base url (to make it absolute)
            img_url = urljoin(url, img_url)
            # Use the basename of the URL (the last part after /) as the filename
            filename = os.path.join("pic", os.path.basename(img_url))
            # Send a GET request to download the image
            img_data = requests.get(img_url).content
            with open(filename, "wb") as f:
                f.write(img_data)


# こびとずかん クレイピング

In [12]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import os
import requests
from urllib.parse import urljoin
import time

# Function to save images and text
def save_images_and_text(soup, url):
    # Save text data
    text = soup.get_text()
    with open(os.path.join("name", f"{url.replace('/', '_')}_text.txt"), "w", encoding='utf-8') as f:
        f.write(text)

    # Save image data
    for img in soup.find_all("img"):
        img_url = img.attrs.get("src")
        if img_url is None:
            continue
        if ".jpg" in img_url or ".webp" in img_url or ".png" in img_url:
            print(f"Saving image from URL: {img_url}")  # URLを出力
            # If the image url is relative, prepend the base url (to make it absolute)
            img_url = urljoin(url, img_url)
            # Use the basename of the URL (the last part after /) as the filename
            # Remove invalid characters from filename
            filename = os.path.join("pic", os.path.basename(img_url).split("?")[0])
            # Send a GET request to download the image
            img_data = requests.get(img_url).content
            with open(filename, "wb") as f:
                f.write(img_data)

# Ensure directories exist
if not os.path.isdir("pic"):
    os.makedirs("pic")
if not os.path.isdir("name"):
    os.makedirs("name")

# URLs to scrape
urls = ["https://wikiwiki.jp/kobito/", "https://www.kobitos.com/kobitos"]

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    save_images_and_text(soup, url)

# For Google Image Search using Selenium
driver = webdriver.Firefox(executable_path='C:\geckodriver\geckodriver.exe')
driver.get("https://www.google.com/")

# Enter the search query and hit Enter
search_box = driver.find_element_by_name("q")
search_box.send_keys("こびとずかん")
search_box.send_keys(Keys.RETURN)

# Click on the 'Images' link to go to the image search results
images_link = driver.find_element_by_link_text("画像")
images_link.click()

# Scroll to load more images
for _ in range(10):  # Adjust the range as needed
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(2)  # Wait for more images to load

soup = BeautifulSoup(driver.page_source, "html.parser")
save_images_and_text(soup, "https://www.google.com/search?tbm=isch&q=%E3%81%93%E3%81%B3%E3%81%A8%E3%81%A5%E3%81%8B%E3%82%93")

driver.quit()


Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/list.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/recentchanges.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/diff.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/backup.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/unfreeze.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/copy.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/rename.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/help.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/plus/home.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/recentchanges.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/new.png?v=4
Saving image from URL: https://cdn.wikiwiki.jp/to/w/common/image/edit.png?v=4
Saving image from URL: https://cdn

In [26]:
from PIL import Image, ImageDraw, ImageFont
import os
import random

# ユーザがこびとの名前を入力
kobito_name = input("こびとの名前を入力してください: ")

# "pic"フォルダ内の全ての画像ファイル名を取得
image_files = [f for f in os.listdir("pic") if f.endswith(('.jpg', '.webp', '.png'))]

if not image_files:
    print("画像が見つかりませんでした。")
else:
    # ランダムに2つの画像を選択
    img1_path = os.path.join("pic", random.choice(image_files))
    img2_path = os.path.join("pic", random.choice(image_files))

    img1 = Image.open(img1_path)
    img2 = Image.open(img2_path)

    # 2つの画像を同じサイズにリサイズ
    img1 = img1.resize((320, 320))
    img2 = img2.resize((320, 320))

    # 2つの画像を50%でブレンド
    img1 = img1.convert("RGB")
    img2 = img2.convert("RGB")
    blended = Image.blend(img1, img2, alpha=0.5)


# テキストを追加する
draw = ImageDraw.Draw(blended)

try:
    font = ImageFont.truetype("meiryo.ttc", 15)  # メイリオフォントとサイズを指定
except OSError:
    font = ImageFont.load_default()  # システムのデフォルトフォントを使用

draw.text((10, 10), kobito_name, fill="black", font=font)  # テキストの位置、内容、色、フォントを指定

# ブレンドした画像を表示
blended.show()



こびとの名前を入力してください:  ももこびと
